## Model:QA Calls For Evaluation
Logging every call to the chain for later review/analysis

In [1]:
%pip install langchain langsmith -y
%pip cache purge

Note: you may need to restart the kernel to use updated packages.



Usage:   
  c:\Users\Mercy\.pyenv\pyenv-win\versions\3.10.11\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  c:\Users\Mercy\.pyenv\pyenv-win\versions\3.10.11\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  c:\Users\Mercy\.pyenv\pyenv-win\versions\3.10.11\python.exe -m pip install [options] [-e] <vcs project url> ...
  c:\Users\Mercy\.pyenv\pyenv-win\versions\3.10.11\python.exe -m pip install [options] [-e] <local project path> ...
  c:\Users\Mercy\.pyenv\pyenv-win\versions\3.10.11\python.exe -m pip install [options] <archive url/path> ...

no such option: -y


Files removed: 0 (0 bytes)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sys
print("Python:", sys.executable)

try:
    import langchain
    import langsmith
    print("langchain:", langchain.__version__)
    print("langsmith:", langsmith.__version__)
except Exception as e:
    print("Import error:", e)

Python: c:\Users\Mercy\.pyenv\pyenv-win\versions\3.10.11\python.exe
langchain: 0.3.26
langsmith: 0.4.4


In [2]:
import langchain.callbacks
print(dir(langchain.callbacks))

['Any', 'AsyncIteratorCallbackHandler', 'DEPRECATED_LOOKUP', 'FileCallbackHandler', 'FinalStreamingStdOutCallbackHandler', 'LangChainTracer', 'StdOutCallbackHandler', 'StreamingStdOutCallbackHandler', 'TYPE_CHECKING', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__getattr__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_import_attribute', 'collect_runs', 'create_importer', 'streaming_aiter', 'streaming_stdout_final_only', 'tracing_enabled', 'tracing_v2_enabled']


In [1]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
from langchain import HuggingFaceHub
from langchain_community.embeddings import HuggingFaceEmbeddings
from dotenv import load_dotenv
import os

In [2]:
# Build your chain
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
faiss_index = FAISS.load_local("../faiss_store", embedding_model, allow_dangerous_deserialization=True)
retriever = faiss_index.as_retriever(search_kwargs={"k": 5})
llm = OpenAI(temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"))

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

response = qa_chain({"query": "How do I create a catalog item in ServiceNow?"})
print(response)

C:\Users\Mercy\AppData\Local\Temp\ipykernel_3504\2422398918.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
C:\Users\Mercy\AppData\Local\Temp\ipykernel_3504\2422398918.py:13: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain({"query": "How do I create a catalog item in ServiceNow?"})


{'query': 'How do I create a catalog item in ServiceNow?', 'result': ' To create a catalog item in ServiceNow, you can follow these steps:\n\n1. Log in to your ServiceNow instance and navigate to the Service Catalog module.\n2. Click on the "Catalog Items" tab.\n3. Click on the "New" button to create a new catalog item.\n4. Enter a name and description for the catalog item.\n5. Select the category and subcategory for the item.\n6. Choose the type of item (e.g. request, order, or task).\n7. Configure the item\'s variables and options.\n8. Add any attachments or images to the item.\n9. Set the pricing and availability for the item.\n10. Save the catalog item.\n\nYou can also add additional features and options to the catalog item, such as approvals, workflows, and fulfillment tasks. Once the item is saved, it will be available for users to request through the Service Catalog.', 'source_documents': [Document(id='7f4b20e5-2113-4a3a-89b6-7b41fbcdd1c1', metadata={'chunk_id': 'chunk_127', 'vi

In [7]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

hf_pipe = pipeline("text2text-generation", model="google/flan-t5-small")
hf_llm = HuggingFacePipeline(pipeline=hf_pipe)
print(hf_llm("What is ITSM in ServiceNow?"))

config.json: 0.00B [00:00, ?B/s]

c:\Users\Mercy\.pyenv\pyenv-win\versions\3.10.11\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Mercy\.cache\huggingface\hub\models--google--flan-t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
C:\Users\Mercy\AppData\Local\Temp\ipykernel_3504\1341218130.py:5: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  hf_llm = HuggingFacePipeline(pipeline=hf_pipe)


ITSM


In [ ]:
chains = {
    "openai-gpt-3.5": RetrievalQA.from_chain_type(llm=llm,  retriever=retriever),
    "flan-t5-small":  RetrievalQA.from_chain_type(llm=hf_llm, retriever=retriever),
}

for name, chain in chains.items():
    print(f"Running: {name}")
    response = chain({"query": "What is ITSM in ServiceNow?"})
    print(response)

Running: openai-gpt-3.5
{'query': 'What is ITSM in ServiceNow?', 'result': ' ITSM stands for Information Technology Service Management and it is a set of processes and tools used to manage and deliver IT services to customers. In ServiceNow, ITSM refers to the suite of products and solutions designed to help organizations manage their IT services and operations.'}
Running: flan-t5-small


Token indices sequence length is longer than the specified maximum sequence length for this model (1035 > 512). Running this sequence through the model will result in indexing errors


{'query': 'What is ITSM in ServiceNow?', 'result': 'ITSM'}


In [ ]:
test_set = [
    ("What is ITSM in ServiceNow?", "ITSM"),
    ("How do I create a catalog item in ServiceNow?", "You can create a catalog item by..."),
]
for q, a_true in test_set:
    resp = qa_chain({"query": q})
    a_pred = resp["result"]
    em = int(a_pred.strip().lower() == a_true.strip().lower())
    print(f"Q: {q}\nPred: {a_pred}\nTrue: {a_true}\nEM: {em}\n")

Q: What is ITSM in ServiceNow?
Pred:  ITSM stands for Information Technology Service Management and it is a set of processes and tools used to manage and deliver IT services to customers. In ServiceNow, ITSM refers to the suite of products and solutions designed to help organizations manage their IT services and operations.
True: ITSM
EM: 0

Q: How do I create a catalog item in ServiceNow?
Pred:  To create a catalog item in ServiceNow, you can follow these steps:

1. Log in to your ServiceNow instance and navigate to the Service Catalog module.
2. Click on the "Catalog Items" tab.
3. Click on the "New" button to create a new catalog item.
4. Enter a name and description for the catalog item.
5. Select the category and subcategory for the item.
6. Choose the type of item (e.g. request, order, or task).
7. Configure the item's variables and options.
8. Add any attachments or images to the item.
9. Set the pricing and availability for the item.
10. Save the catalog item.

You can also add